In [2]:
print("Zomato analytics project")

Zomato analytics project


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rabhar/zomato-restaurants-in-india")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/zomato-restaurants-in-india


Restaurant analytics can be very useful for companies like Swiggy and Zomato. It helps them understand which restaurants are in high demand, which cuisines are more preferred in different cities, and how much revenue different locations contribute. Analytics also shows how wide their delivery coverage is across various pin codes and how customers are responding in different regions.

Based on these insights, they can make smarter business decisions such as:


* optimizing recommendations
* improving delivery coverage
* designing city-specific marketing strategies
* enhancing customer experience
* helping restaurants grow through data-driven insights

In this project, I explore restaurant data to uncover meaningful insights about demand, pricing, cuisines, ratings, and city-wise behavior.

**Dataset**
For this analysis, I have used the **Zomato Restaurants in India** dataset from Kaggle.  
It contains detailed information about restaurants across various Indian cities, including:
- Restaurant Name  
- City / Location  
- Cuisines  
- Ratings and Votes  
- Average Cost  
- Delivery and Dining Details

  Dataset Source: https://www.kaggle.com/datasets/rabhar/zomato-restaurants-in-india


In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
#Seaborn style setting
sns.set(style="whitegrid")


In [6]:
import os
os.listdir('/kaggle/input/zomato-restaurants-in-india')

['zomato_restaurants_in_India.csv']

In [7]:
#Load Data
df = pd.read_csv("/kaggle/input/zomato-restaurants-in-india/zomato_restaurants_in_India.csv")
df.head()
df.shape
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211944 entries, 0 to 211943
Data columns (total 26 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   res_id                211944 non-null  int64  
 1   name                  211944 non-null  object 
 2   establishment         211944 non-null  object 
 3   url                   211944 non-null  object 
 4   address               211810 non-null  object 
 5   city                  211944 non-null  object 
 6   city_id               211944 non-null  int64  
 7   locality              211944 non-null  object 
 8   latitude              211944 non-null  float64
 9   longitude             211944 non-null  float64
 10  zipcode               48757 non-null   object 
 11  country_id            211944 non-null  int64  
 12  locality_verbose      211944 non-null  object 
 13  cuisines              210553 non-null  object 
 14  timings               208070 non-null  object 
 15  

In [8]:
df.columns


Index(['res_id', 'name', 'establishment', 'url', 'address', 'city', 'city_id',
       'locality', 'latitude', 'longitude', 'zipcode', 'country_id',
       'locality_verbose', 'cuisines', 'timings', 'average_cost_for_two',
       'price_range', 'currency', 'highlights', 'aggregate_rating',
       'rating_text', 'votes', 'photo_count', 'opentable_support', 'delivery',
       'takeaway'],
      dtype='object')

**Data Cleaning**

**Making sure datatypes are of consistent nature**

In [9]:

df.dtypes

res_id                    int64
name                     object
establishment            object
url                      object
address                  object
city                     object
city_id                   int64
locality                 object
latitude                float64
longitude               float64
zipcode                  object
country_id                int64
locality_verbose         object
cuisines                 object
timings                  object
average_cost_for_two      int64
price_range               int64
currency                 object
highlights               object
aggregate_rating        float64
rating_text              object
votes                     int64
photo_count               int64
opentable_support       float64
delivery                  int64
takeaway                  int64
dtype: object

Although the delivery column is stored as int64, it represents a categorical feature indicating delivery availability. The values −1, 0, and 1 correspond to unavailable, unknown, and available states respectively. Since these values do not represent numerical magnitude or ordering, the column is treated as categorical rather than numeric.


The original `delivery` column was retained in its encoded form (-1, 0, 1) to avoid data corruption during iterative EDA.
A separate categorical column (`delivery_label`) was created for interpretability in analysis and visualizations.


In [10]:
df['delivery'].value_counts()


delivery
-1    132573
 1     78335
 0      1036
Name: count, dtype: int64

In [11]:
df['delivery_label'] = (
    df['delivery']
    .replace({1: 'Yes', 0: 'Unknown', -1: 'No'})
    .astype('category')
)


In [12]:
#A new column delivery_label has been added
df.columns

Index(['res_id', 'name', 'establishment', 'url', 'address', 'city', 'city_id',
       'locality', 'latitude', 'longitude', 'zipcode', 'country_id',
       'locality_verbose', 'cuisines', 'timings', 'average_cost_for_two',
       'price_range', 'currency', 'highlights', 'aggregate_rating',
       'rating_text', 'votes', 'photo_count', 'opentable_support', 'delivery',
       'takeaway', 'delivery_label'],
      dtype='object')

In [13]:
df.dtypes

res_id                     int64
name                      object
establishment             object
url                       object
address                   object
city                      object
city_id                    int64
locality                  object
latitude                 float64
longitude                float64
zipcode                   object
country_id                 int64
locality_verbose          object
cuisines                  object
timings                   object
average_cost_for_two       int64
price_range                int64
currency                  object
highlights                object
aggregate_rating         float64
rating_text               object
votes                      int64
photo_count                int64
opentable_support        float64
delivery                   int64
takeaway                   int64
delivery_label          category
dtype: object

**Removing irrelevant columns**


The following columns don’t add analytical value for EDA:


* url: This only contains the restaurant’s Zomato webpage link.
* address:The address is a long unstructured text field. Most insights related to location can already be derived from structured fields like city, locality, zipcode
* city_id: This is simply a numeric mapping of the city column. Since the city name itself is already available and more readable, city_id becomes redundant and unnecessary for analysis.
* country_id: This dataset only contains restaurants from India, so the country is the same for all rows.
* locality_verbose:This is a detailed descriptive version of locality, but we already have structured and cleaner locality and city columns.
* photo_count :Dropped because it reflects user engagement rather than restaurant quality or performance and does not contribute meaningful insights to the current analysis.
* opentable_support: Dropped because it is not relevant to the Indian restaurant context and has minimal impact on pricing, ratings, or customer preference analysis.

These columns were dropped because they either serve as identifiers, URLs, redundant information, or unstructured text that does not contribute to meaningful insights in exploratory data analysis.


In [14]:
df = df.drop(columns=['url','city_id','country_id','locality_verbose','address' ,'opentable_support' , 'photo_count' ])


In [15]:
df.columns

Index(['res_id', 'name', 'establishment', 'city', 'locality', 'latitude',
       'longitude', 'zipcode', 'cuisines', 'timings', 'average_cost_for_two',
       'price_range', 'currency', 'highlights', 'aggregate_rating',
       'rating_text', 'votes', 'delivery', 'takeaway', 'delivery_label'],
      dtype='object')

**Handling Missing Values**

In [16]:
df.isnull().sum()


res_id                       0
name                         0
establishment                0
city                         0
locality                     0
latitude                     0
longitude                    0
zipcode                 163187
cuisines                  1391
timings                   3874
average_cost_for_two         0
price_range                  0
currency                     0
highlights                   0
aggregate_rating             0
rating_text                  0
votes                        0
delivery                     0
takeaway                     0
delivery_label               0
dtype: int64

• Removed zipcode because it had a very high percentage of missing values and offered limited analytical value.

• Dropped rows with missing cuisines since cuisine information is essential for analysis.

• Filled missing timings values with “Not Available”.

• Removed opentable_support column as it is not relevant for analysis


In [17]:
df = df.drop(columns=['zipcode'])
df = df.dropna(subset=['cuisines'])
df['timings'] = df['timings'].fillna("Not Available")




In [18]:
df.dtypes

res_id                     int64
name                      object
establishment             object
city                      object
locality                  object
latitude                 float64
longitude                float64
cuisines                  object
timings                   object
average_cost_for_two       int64
price_range                int64
currency                  object
highlights                object
aggregate_rating         float64
rating_text               object
votes                      int64
delivery                   int64
takeaway                   int64
delivery_label          category
dtype: object

**Removing Duplicate**
Since multiple duplicate restaurant entries existed in the dataset, duplicates were identified using the unique res_id column. Only one entry per restaurant was retained and the rest were removed to ensure unbiased and accurate analysis.

In [19]:
df['res_id'].duplicated().sum()


np.int64(155455)

In [20]:
df = df.drop_duplicates(subset=['res_id'])


In [21]:
df.shape

(55098, 19)

In [22]:

df['res_id'].duplicated().sum()

np.int64(0)

**Cleaning ratings**

*  Some restaurants have aggregate_rating = 0.0 but are actually “Not Rated”
*   Some may have rating_text mismatch
*   Some have very few votes → unreliable rating
*    Need to ensure rating is numeric and clean


In [23]:
df['rating_text'].info

<bound method Series.info of 0         Very Good
1         Very Good
2         Very Good
3         Very Good
4         Excellent
            ...    
211882      Average
211925    Very Good
211926         Good
211940    Very Good
211942         Good
Name: rating_text, Length: 55098, dtype: object>

The rating_text column was dropped because it is simply a textual representation of the aggregate_rating column. Since aggregate_rating already provides a more precise and useful numerical value for analysis, keeping both would be redundant.

In [24]:
df = df.drop(columns = 'rating_text')

In [25]:
df.columns

Index(['res_id', 'name', 'establishment', 'city', 'locality', 'latitude',
       'longitude', 'cuisines', 'timings', 'average_cost_for_two',
       'price_range', 'currency', 'highlights', 'aggregate_rating', 'votes',
       'delivery', 'takeaway', 'delivery_label'],
      dtype='object')

In [26]:
df['aggregate_rating'].info

<bound method Series.info of 0         4.4
1         4.4
2         4.2
3         4.3
4         4.9
         ... 
211882    2.9
211925    4.0
211926    3.9
211940    4.1
211942    3.7
Name: aggregate_rating, Length: 55098, dtype: float64>

In [27]:
df['aggregate_rating'].unique()

array([4.4, 4.2, 4.3, 4.9, 4. , 3.8, 3.4, 4.1, 3.5, 4.6, 3.9, 3.6, 4.5,
       4.7, 3.7, 4.8, 3.2, 0. , 3.3, 2.8, 3.1, 2.6, 3. , 2.7, 2.9, 2.2,
       2.3, 2.4, 2.5, 2.1, 1.8, 2. , 1.9])

In [28]:
df['aggregate_rating'].describe()

count    55098.000000
mean         2.979478
std          1.449025
min          0.000000
25%          2.900000
50%          3.500000
75%          3.900000
max          4.900000
Name: aggregate_rating, dtype: float64

In [29]:
#Check how many times rating 0 appears
(df['aggregate_rating'] == 0).sum()


np.int64(9662)

Restaurants with 0.0 rating represent unrated restaurants. Since they do not contribute meaningful information for rating analysis, they were removed.

In [30]:
df = df[df['aggregate_rating'] > 0]

In [31]:
df.shape

(45436, 18)

To improve the reliability of rating-based analysis, only restaurants with at least 5 votes were retained. Ratings with very few votes may not be representative of overall customer sentiment and can introduce noise into the analysis.

In [32]:
df['votes'].info

<bound method Series.info of 0          814
1         1203
2          801
3          693
4          470
          ... 
211882       4
211925     111
211926     207
211940     187
211942     128
Name: votes, Length: 45436, dtype: int64>

In [33]:
df = df[df['votes'] >= 5]

In [34]:
df.shape

(43149, 18)

In [35]:
df.dtypes

res_id                     int64
name                      object
establishment             object
city                      object
locality                  object
latitude                 float64
longitude                float64
cuisines                  object
timings                   object
average_cost_for_two       int64
price_range                int64
currency                  object
highlights                object
aggregate_rating         float64
votes                      int64
delivery                   int64
takeaway                   int64
delivery_label          category
dtype: object

In [36]:
df['price_range'].value_counts()

price_range
1    19695
2    14262
3     6623
4     2569
Name: count, dtype: int64

**The price_range column has four unique values. It was treated as a categorical variable, where 1 represents budget-friendly restaurants and 4 represents expensive restaurants.**

In [37]:
df['price_range'] = df['price_range'].astype('category')

In [38]:
df['price_range'].dtypes

CategoricalDtype(categories=[1, 2, 3, 4], ordered=False, categories_dtype=int64)

In [82]:
df.dtypes

res_id                     int64
name                      object
establishment             object
city                      object
locality                  object
latitude                 float64
longitude                float64
cuisines                  object
timings                   object
average_cost_for_two       int64
price_range             category
currency                  object
highlights                object
aggregate_rating         float64
votes                      int64
delivery                   int64
takeaway                   int64
delivery_label          category
dtype: object

**avg_cost_for_two**

In [39]:
df['average_cost_for_two'].describe()

count    43149.000000
mean       592.702380
std        639.994149
min          0.000000
25%        250.000000
50%        400.000000
75%        700.000000
max      30000.000000
Name: average_cost_for_two, dtype: float64

99th percentile is the value below which 99% of the data lies

In [40]:
upper_limit = df['average_cost_for_two'].quantile(0.99)

The 99th percentile value of average_cost_for_two was calculated and used as an upper limit. Restaurants with an average cost for two exceeding this threshold were removed to eliminate extreme outliers and improve the interpretability of the analysis.

In [41]:
upper_limit

np.float64(3000.0)

In [42]:
df = df[df['average_cost_for_two'] <= upper_limit]

In [43]:
df['average_cost_for_two'].describe()

count    42779.000000
mean       558.951004
std        495.699505
min          0.000000
25%        250.000000
50%        400.000000
75%        700.000000
max       3000.000000
Name: average_cost_for_two, dtype: float64

**Working on currency**

In [44]:
df['currency'].value_counts()

currency
Rs.    42779
Name: count, dtype: int64

All the values in currency Have same value Rs. So it is better to delete this column

In [45]:
df = df.drop(columns = ['currency'])

In [96]:
df.dtypes

res_id                     int64
name                      object
establishment             object
city                      object
locality                  object
latitude                 float64
longitude                float64
cuisines                  object
timings                   object
average_cost_for_two       int64
price_range             category
highlights                object
aggregate_rating         float64
votes                      int64
delivery                   int64
takeaway                   int64
delivery_label          category
dtype: object

**City columns : Making city names consistent**

In [46]:
df['city'].unique()

array(['Agra', 'Ahmedabad', 'Gandhinagar', 'Ajmer', 'Alappuzha',
       'Allahabad', 'Amravati', 'Amritsar', 'Aurangabad', 'Bangalore',
       'Bhopal', 'Bhubaneshwar', 'Chandigarh', 'Mohali', 'Panchkula',
       'Zirakpur', 'Nayagaon', 'Chennai', 'Coimbatore', 'Cuttack',
       'Darjeeling', 'Dehradun', 'New Delhi', 'Gurgaon', 'Noida',
       'Faridabad', 'Ghaziabad', 'Greater Noida', 'Dharamshala',
       'Gangtok', 'Goa', 'Gorakhpur', 'Guntur', 'Guwahati', 'Gwalior',
       'Haridwar', 'Hyderabad', 'Secunderabad', 'Indore', 'Jabalpur',
       'Jaipur', 'Jalandhar', 'Jammu', 'Jamnagar', 'Jamshedpur', 'Jhansi',
       'Jodhpur', 'Junagadh', 'Kanpur', 'Kharagpur', 'Kochi', 'Kolhapur',
       'Kolkata', 'Howrah', 'Kota', 'Lucknow', 'Ludhiana', 'Madurai',
       'Manali', 'Mangalore', 'Manipal', 'Udupi', 'Meerut', 'Mumbai',
       'Thane', 'Navi Mumbai', 'Mussoorie', 'Mysore', 'Nagpur',
       'Nainital', 'Nasik', 'Nashik', 'Neemrana', 'Ooty', 'Palakkad',
       'Patiala', 'Patna', 'Pudu

Real-world datasets often contain inconsistencies in text fields due to multiple naming conventions and spellings. Therefore, manual inspection was performed to identify such variations, and domain knowledge was used to standardize city names for accurate analysis.

There were a total of 96 unique city names in the dataset. While this is not extremely large, manual inspection alone is still prone to human error and inconsistency.

To reduce noise and standardize text before comparison, an initial normalization step was applied where city names were:

converted to lowercase

stripped of leading and trailing whitespaces

deduplicated

This normalization ensures that superficial formatting differences do not affect similarity matching. After normalization, string similarity techniques (difflib) were used during EDA to identify potential duplicate or misspelled city names for further review.

In [49]:
# Derive a normalized list of city names for analysis without modifying the original DataFrame,
# preserving raw data integrity during exploratory analysis
cities = df['city'].dropna().str.lower().str.strip().unique()

In [51]:
from difflib import SequenceMatcher

def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

similar_pairs = []

for i in range(len(cities)):
    for j in range(i + 1, len(cities)):
        score = similarity(cities[i], cities[j])
        if score > 0.85 and cities[i] != cities[j]:
            similar_pairs.append((cities[i], cities[j], score))

In [53]:
similar_pairs

[('bangalore', 'mangalore', 0.8888888888888888),
 ('nasik', 'nashik', 0.9090909090909091)]

In [57]:
city_mapping = {
   
    'nasik': 'nashik'
}


In [60]:
df['city_cleaned'] = (
    df['city']
    .str.lower()
    .str.strip()
    .replace(city_mapping)
)

In [61]:
df.columns

Index(['res_id', 'name', 'establishment', 'city', 'locality', 'latitude',
       'longitude', 'cuisines', 'timings', 'average_cost_for_two',
       'price_range', 'highlights', 'aggregate_rating', 'votes', 'delivery',
       'takeaway', 'delivery_label', 'city_cleaned'],
      dtype='object')